<a href="https://colab.research.google.com/github/toussaintma/petitproust/blob/main/pp_retrieve_hflc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain
!pip install huggingface_hub
!pip install sentence-transformers
!pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 867.6/867.6 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.4/116.4 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.7/142.7 kB 2.3 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 3.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cache

In [ ]:
import locale

locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
# Retriever section

In [ ]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("./input.txt")

In [ ]:
docs = loader.load()
len(docs)

1

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=30)

chunked_docs = splitter.split_documents(docs)

In [ ]:
len(chunked_docs)

16459

In [ ]:
chunked_docs[0:2]

[Document(page_content='MARCEL PROUST\n\nÀ la RECHERCHE DU TEMPS PERDU\n\nTOME I\n\nDu Côté de Chez Swann\n\n\n\n\nÀ Monsieur Gaston Calmette\n\n_Comme un témoignage de profonde\net affectueuse reconnaissance_,\n\nMarcel Proust.\n\n\n\n\nPREMIÈRE PARTIE\nCOMBRAY\n\n\nI.', metadata={'source': './input.txt'}),
 Document(page_content='Longtemps, je me suis couché de bonne heure. Parfois, à peine ma\nbougie éteinte, mes yeux se fermaient si vite que je n’avais pas le\ntemps de me dire: «Je m’endors.» Et, une demi-heure après, la pensée\nqu’il était temps de chercher le sommeil m’éveillait; je voulais poser\nle volume que je croyais avoir encore dans les mains et souffler ma\nlumière; je n’avais pas cessé en dormant de faire des réflexions sur\nce que je venais de lire, mais ces réflexions avaient pris un tour un', metadata={'source': './input.txt'}),
 Document(page_content='peu particulier; il me semblait que j’étais moi-même ce dont parlait\nl’ouvrage: une église, un quatuor, la rivalité 

In [ ]:
# BAAI/bge-m3 maybe try sentence-transformers/all-MiniLM-L12-v2
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

db = FAISS.from_documents(chunked_docs[0:50], HuggingFaceEmbeddings(model_name="BAAI/bge-m3"))

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/15.0k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/687 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [ ]:
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 4})

In [ ]:
retriever.invoke("endormissement")

[Document(page_content='Je me rendormais, et parfois je n’avais plus que de courts réveils\nd’un instant, le temps d’entendre les craquements organiques des\nboiseries, d’ouvrir les yeux pour fixer le kaléidoscope de\nl’obscurité, de goûter grâce à une lueur momentanée de conscience le\nsommeil où étaient plongés les meubles, la chambre, le tout dont je\nn’étais qu’une petite partie et à l’insensibilité duquel je retournais\nvite m’unir. Ou bien en dormant j’avais rejoint sans effort un âge à', metadata={'source': './input.txt'}),
 Document(page_content='Longtemps, je me suis couché de bonne heure. Parfois, à peine ma\nbougie éteinte, mes yeux se fermaient si vite que je n’avais pas le\ntemps de me dire: «Je m’endors.» Et, une demi-heure après, la pensée\nqu’il était temps de chercher le sommeil m’éveillait; je voulais poser\nle volume que je croyais avoir encore dans les mains et souffler ma\nlumière; je n’avais pas cessé en dormant de faire des réflexions sur\nce que je venais de lir

In [ ]:
from huggingface_hub import login
from google.colab import userdata

login(token=userdata.get('HF_TOKEN'))

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
# model
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

In [ ]:
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from transformers import pipeline
from langchain_core.output_parsers import StrOutputParser

text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.2,
    do_sample=True,
    repetition_penalty=1.1,
    return_full_text=True,
    max_new_tokens=500,
)

llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

prompt_template = """
<|system|>

You are an expert writer writing in french. Rewrite this text in the litterary style of author Marcel Proust. Use the following example texts of his writing to help:

{examples}

</s>
<|user|>

{source}

</s>
<|assistant|>

 """

prompt = PromptTemplate(
    input_variables=["examples", "source"],
    template=prompt_template,
)

llm_chain = prompt | llm | StrOutputParser()

In [ ]:
from langchain_core.runnables import RunnablePassthrough

retriever = db.as_retriever()

rag_chain = {"examples": retriever, "source": RunnablePassthrough()} | llm_chain

In [ ]:
original_text = """
Chaque soir je m'endors en rêvant de fusée. Dans mes rêves je suis un cosmonaute dans une fusée partant pour la lune et je rêve.
"""
answer_llmonly = llm_chain.invoke({"examples": "", "source": original_text})
answer_rag = rag_chain.invoke(original_text)

In [ ]:
print(answer_llmonly)


<|system|>

You are an expert writer writing in french. Rewrite this text in the litterary style of author Marcel Proust. Use the following example texts of his writing to help:



</s>
<|user|>


Chaque soir je m'endors en rêvant de fusée. Dans mes rêves je suis un cosmonaute dans une fusée partant pour la lune et je rêve.


</s>
<|assistant|>

  Every night, I dream of a rocket launching into space. In my dreams, I am a cosmonaut in a spaceship heading towards the moon and I dream.

  As I drift off to sleep, I imagine myself soaring through the heavens, far beyond the stars. My mind is filled with images of the vast expanse of space, the infinite beauty of the universe.

  And as I fall asleep, I close my eyes and let myself be carried away by the rhythmic beat of my heart, by the thrill of the unknown, by the power of the cosmos.

  In my dreams, I am not just a mere observer, but a participant in this grand adventure. I am a part of something bigger than myself, something that tr

In [ ]:
print(answer_rag)


<|system|>

You are an expert writer writing in french. Rewrite this text in the litterary style of author Marcel Proust. Use the following example texts of his writing to help:

[Document(page_content='Longtemps, je me suis couché de bonne heure. Parfois, à peine ma\nbougie éteinte, mes yeux se fermaient si vite que je n’avais pas le\ntemps de me dire: «Je m’endors.» Et, une demi-heure après, la pensée\nqu’il était temps de chercher le sommeil m’éveillait; je voulais poser\nle volume que je croyais avoir encore dans les mains et souffler ma\nlumière; je n’avais pas cessé en dormant de faire des réflexions sur\nce que je venais de lire, mais ces réflexions avaient pris un tour un', metadata={'source': './input.txt'}), Document(page_content='(comme l’hirondelle de mer qui a son nid au fond d’un souterrain dans\nla chaleur de la terre), et où, le feu étant entretenu toute la nuit\ndans la cheminée, on dort dans un grand manteau d’air chaud et fumeux,\ntraversé des lueurs des tisons qui 